In [1529]:
import pandas as pd
import numpy as np 
from category_encoders import BinaryEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [1530]:
#load the data and find categories
posts_df = pd.read_csv('posts.csv')
users_df = pd.read_csv('users.csv')
views_df = pd.read_csv('views.csv')
posts_df['category'].unique()

array(['Plant Biotechnology',
       'Artificial Intelligence|Machine Learning|Information Technology',
       'Operating Systems', 'Drawings', 'Competition Laws', 'Eco System',
       'Economic Policies', 'Graphic|Graphic Design', 'Painting',
       'Pen and ink', 'Computer Technology|Information Technology',
       'Drawings|Painting',
       'Graphic Design|Visual Arts|Illustration|Graphic',
       'Drawings|Calligraphy', 'Photography', 'Empowerment', nan,
       'Video editing', 'Inorganic Chemistry', 'Programming languages',
       'Conceptual|Graphic Design', 'HR Management',
       'Human Resources|HR Management',
       'Mass Media|International Relations', 'Sculptures|Artistic design',
       'Fashion Design|Ceramics|Artistic design', 'Craft|Artistic design',
       'Fashion Design|Visual Arts|Conceptual|Artistic design',
       'Photography|Fashion Design|Visual Arts|Graphic Design|Artistic design',
       'Fashion Design|Visual Arts|Graphic Design|Artistic design|Graphic|Ill

In [1531]:
#This will be a lookup table for the engine
postviewer = pd.DataFrame()
postviewer['user_id'] = views_df['user_id']
postviewer['post_id'] = views_df['post_id']


In [1532]:
categories = posts_df['category'].unique()
types = [posts_df[' post_type'].unique()]
posts_df['category'].isnull().sum().sum()

28

In [1533]:
#Encoding category
encoder = BinaryEncoder(cols=['category'])
newcat = encoder.fit_transform(posts_df['category'])
posts_df = pd.concat([posts_df, newcat], axis = 1)
posts_df

,_id,title,category,post_type,category_0,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog,0,0,0,0,0,0,0,0,1
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog,0,0,0,0,0,0,0,1,0
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog,0,0,0,0,0,0,0,1,1
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork,0,0,0,0,0,0,1,0,0
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,5ed3791976027d35905ccab6,CI/CD using GitHub Actions,Computer Technology|Information Technology,blog,0,0,0,0,0,1,0,1,1
489,5ed3820f76027d35905ccac8,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology|Computer Application|Infor...,blog,0,0,1,0,0,1,0,1,0
490,5ed3ef4cbd514d602c1530f2,Recommendation Engine,NaN,project,0,0,0,0,1,0,0,0,1
491,5ed415c6bd514d602c15312d,Face Recognition using Transfer Learning.,Science; Technology,skill,0,1,1,1,0,1,0,0,0


In [1534]:
# enc = BinaryEncoder()
types = posts_df[' post_type'].unique()
# newtype = enc.fit_transform(posts_df[[' post_type']])
# posts_df = pd.concat([posts_df, newtype], axis = 1)
# posts_df
types

# #blog:001
# #artwork: 010
# #project: 011
# #skill: 100

array(['blog', 'artwork', 'project', 'skill'], dtype=object)

In [1535]:
posts_df['category_0'] = posts_df['category_0'].astype('str')
posts_df['category_1'] = posts_df['category_1'].astype('str')
posts_df['category_2'] = posts_df['category_2'].astype('str')
posts_df['category_3'] = posts_df['category_3'].astype('str')
posts_df['category_4'] = posts_df['category_4'].astype('str')
posts_df['category_5'] = posts_df['category_5'].astype('str')
posts_df['category_6'] = posts_df['category_6'].astype('str')
posts_df['category_7'] = posts_df['category_7'].astype('str')
posts_df['category_8'] = posts_df['category_8'].astype('str')

In [1536]:
posts_df['cate'] = posts_df['category_0'] + posts_df['category_1'] + posts_df['category_2'] + posts_df['category_3']+ posts_df['category_4'] + posts_df['category_5'] + posts_df['category_6'] + posts_df['category_7'] + posts_df['category_8']
# pd.to_numeric(posts_df['cate'])

In [1537]:
# posts_df[' post_type_0'] = posts_df[' post_type_0'].astype('int')
# posts_df[' post_type_1'] = posts_df[' post_type_1'].astype('int')
# posts_df[' post_type_2'] = posts_df[' post_type_2'].astype('int')

In [1538]:
# posts_df['types'] =  posts_df[' post_type_0'] + posts_df[' post_type_1'] + posts_df[' post_type_2']

In [1539]:
to_drop = ['category_0', 'category_1', 'category_2', 'category_3' ,'category_4', 'category_5', 'category_6', 'category_7', 'category_8']
posts_df.drop(to_drop, inplace=True, axis=1)


In [1540]:
views_df.drop('timestamp', inplace=True, axis=1)

In [1541]:
# views_df['category'] = ''
postcat = dict(zip(posts_df['_id'], posts_df['cate']))


for cols in views_df.columns:
    print(cols)


# post_id = views_df['post_id'].values.tolist()
# post_id1 = views_df['post_id'].tolist()

# post_id1
postcat

user_id
post_id


001011',
 '5db2a7d099cbb90e4339c734': '000001100',
 '5db2b91a99cbb90e4339c749': '000000101',
 '5dbc622a99cbb90e4339c7f6': '000001101',
 '5dbc631f99cbb90e4339c7fd': '000001110',
 '5dc065ca24b883670268772f': '000001111',
 '5dd1751db802e41ed198b680': '000010000',
 '5dde6a91369b28584ecca156': '000001111',
 '5ddeb6e80eb5e25a8a07f065': '000010001',
 '5de179d80eb5e25a8a07f079': '000010001',
 '5de7971b8eab6401affbb137': '000000100',
 '5de8d73249e8203ff9219a74': '000010010',
 '5dea816a42a8854bf6eaba89': '000010011',
 '5dee9b5042a8854bf6eabaaf': '000010001',
 '5df64ad1cc709107827fb3dd': '000000100',
 '5e04fb252a37d20505da2a2b': '000010100',
 '5e1029f22a37d20505da2a79': '000010101',
 '5e27002d2a37d20505da2c13': '000010110',
 '5e281cf0025ae20886de2050': '000000100',
 '5e2867f6025ae20886de205a': '000010111',
 '5e294fd1c85ab714a7da6643': '000011000',
 '5e2d36ebc85ab714a7da66d1': '000011001',
 '5e2d3fbfc85ab714a7da66d5': '000011010',
 '5e2d447dc85ab714a7da66d7': '000011011',
 '5e2d4737c85ab714a7da66d

In [1542]:
# pickle_out = open("postcat.pickle","wb")
# pickle.dump(posts_df,pickle_out)
# pickle_out.close()

In [1543]:
list1 = []
postid = views_df['post_id'].tolist()
for i in range(len(postid)):
    try:
        postid[i]= postcat[postid[i]]
    except KeyError:
        list1.append(postid[i])

views_df['Category'] = postid

In [1544]:


views_df['Category'] 

0       011011000
1       001001010
2       000001111
3       001011111
4       001001010
          ...    
1444    000000100
1445    001101011
1446    000000100
1447    001101111
1448    000010101
Name: Category, Length: 1449, dtype: object

In [1545]:
keys=[]
try:
    views_df['Category'] = views_df['Category'].astype(int)
except ValueError:
    keys.append('key')

In [1546]:
keys

['key']

In [1547]:
#Encoding genders
le = LabelEncoder()
pret = users_df['gender'].unique()
users_df['gender'] = le.fit_transform(users_df['gender'])
users_df['gender'].astype(int)

0      1
1      1
2      1
3      1
4      1
      ..
113    1
114    1
115    1
116    1
117    0
Name: gender, Length: 118, dtype: int32

In [1548]:
#custom function to apply encoding to all columns using dicts
def convert(ids, applieddict):
    for i in range(len(ids)):
        try:
            ids[i]= applieddict[ids[i]]
        except KeyError:
            listuse.append(ids[i])
    return ids

In [1549]:
# le1 = LabelEncoder()
acadict = dict({'undefined': 0, 'undergraduate': 1, 'graduate':2})
useracads = users_df['academics'].tolist()
users_df['academicsenc'] = convert(useracads, acadict)
useracads1 = users_df['academicsenc'].tolist
pre = users_df['academics'].unique()

print(pre)
print(users_df['academicsenc'].unique())
# print(pre)
# print(users_df['academics'].unique())
#labels 
# 2 : Graduate
# 1 : undergrad
#0: undefined


['undergraduate' 'graduate' 'undefined']
[1 2 0]


In [1550]:
listuse = []

usergender = dict(zip(users_df['_id'], users_df['gender']))
usertype = dict({'project': 4, 'blog': 3, 'skill':2, 'artwork':1})
#numbers assigned based on the amount of time spend on each post type

useracademics = dict(zip(users_df['_id'], users_df['academicsenc']))
postdict = dict(zip(posts_df['_id'], posts_df['title']))
categorydict = dict(zip(posts_df['category'], posts_df['cate']))



In [1551]:
# postid = postviewer['post_id'].tolist()
# postviewer['Category'] = convert(postid, postcat)
# cats = postviewer['Category'].tolist()
# postviewer['Category'] = convert(cats, categorydict)
postid = postviewer['post_id'].tolist()
postviewer['Categoryenc'] = convert(postid, postcat)
categories = postviewer['Categoryenc'].tolist()
postviewer['Categoryenc'] = convert(categories, categorydict)


In [1552]:
pickle_out = open("postviewer.pickle","wb")
pickle.dump(postviewer,pickle_out)
pickle_out.close()
postviewer

,user_id,post_id,Categoryenc
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,011011000
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,001001010
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,000001111
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,001011111
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,001001010
...,...,...,...
1444,5d60098a653a331687083238,5e7a60edcfc8b713f5ac7d82,000000100
1445,5d60098a653a331687083238,5e7a629ccfc8b713f5ac7d84,001101011
1446,5d60098a653a331687083238,5e7b1e65cfc8b713f5ac7d90,000000100
1447,5d60098a653a331687083238,5e7cc074cfc8b713f5ac7db0,001101111


In [1553]:
userid = views_df['user_id'].tolist()
typelist = posts_df[' post_type'].tolist()
posts_df[' post_type'] = convert(typelist, usertype)
postype = dict(zip(posts_df['_id'], posts_df[' post_type']))

In [1554]:
# listuser = []

# for i in range(len(userid)):
#     try:
#        userid[i] = usergender[userid[i]]
#     except KeyError:
#         listuser.append(userid[i])



In [1555]:
userid = views_df['user_id'].tolist()
views_df['gender'] = convert(userid, usergender)
views_df['gender']
#Labels
#female:0
#male:1
#undefined:2

0       0
1       1
2       1
3       1
4       1
       ..
1444    1
1445    1
1446    1
1447    1
1448    1
Name: gender, Length: 1449, dtype: int64

In [1556]:
userid = views_df['user_id'].tolist()
views_df['academics'] = convert(userid, useracademics)
postid = views_df['post_id'].tolist()
postid
views_df['post_type'] = convert(postid,postype)
views_df
# views_df['post_id'] = convert(post_id1, postdict)

# views_df['post_type_0'] = posts_df[' post_type_0'].astype('int')
# views_df['post_type_1'] = posts_df[' post_type_1'].astype('int')
# views_df['post_type_2'] = posts_df[' post_type_2'].astype('int')
views_df.to_csv('checkthis.csv')

In [1557]:

for i in range(len(views_df['post_type'])):
    if views_df.loc[i,'post_type'] == str:
        views_df.loc[i,'post_type'] = 0

In [1558]:
views_df
views_df.drop([748], inplace=True, axis=0)
views_df.drop([1193], inplace=True, axis=0)


In [1559]:
# views_df.loc[1193,'Category'] = '000000000'
# views_df.loc[748,'Category'] = '000000000'
pd.to_numeric(views_df['Category'])

y = views_df['Category'].values

dropit = [ 'post_id', 'user_id']
views_df['Category'].dropna(inplace=True)
views_df.drop(dropit, inplace=True, axis=1)
pd.to_numeric(views_df['gender'])
pd.to_numeric(views_df['academics'])
# views_df.loc[748,'post_type'] = 0
# views_df.loc[1193,'post_type'] =0
pd.to_numeric(views_df['post_type'])
X = views_df
pickle_out = open("views.pickle","wb")
pickle.dump(views_df,pickle_out)
pickle_out.close()

In [1560]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [1561]:
# pickle_out = open("X_train.pickle","wb")
# pickle.dump(X_train,pickle_out)
# pickle_out.close()

# pickle_out = open("X_test.pickle","wb")
# pickle.dump(X_test,pickle_out)
# pickle_out.close()

# pickle_out = open("y_train.pickle","wb")
# pickle.dump(y_train,pickle_out)
# pickle_out.close()

# pickle_out = open("y_test.pickle","wb")
# pickle.dump(y_test,pickle_out)
# pickle_out.close()

views_df

,Category,gender,academics,post_type
0,011011000,0,1,1
1,001001010,1,1,3
2,000001111,1,1,1
3,001011111,1,1,3
4,001001010,1,1,3
...,...,...,...,...
1444,000000100,1,1,1
1445,001101011,1,1,1
1446,000000100,1,1,1
1447,001101111,1,1,3


In [1562]:
#this is the dataset i made to make the similar post recommendation
usertyperev = dict({ 4:'project',  3:'blog', 2 :'skill', 1: 'artwork'})
cb_df = pd.DataFrame(columns = ['title', 'category', 'post_type'])
cb_df['title'] = posts_df['title'] 
cb_df['category'] = posts_df['category'] 
cb_df['post_type'] = convert(typelist, usertyperev)

#just had to do this once


In [1563]:
cb_df

,title,category,post_type
0,hello there,Plant Biotechnology,blog
1,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,What is an Operating System ?,Operating Systems,blog
3,Lord Shiva,Drawings,artwork
4,How Competition law evolved?,Competition Laws,blog
...,...,...,...
488,CI/CD using GitHub Actions,Computer Technology|Information Technology,blog
489,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology|Computer Application|Infor...,blog
490,Recommendation Engine,NaN,project
491,Face Recognition using Transfer Learning.,Science; Technology,skill


In [1564]:
cb_df['features'] = cb_df['title'] +" "+ cb_df['category'] +" "+ cb_df['post_type']
cb_df['features']= cb_df['features'].fillna('')

In [1565]:
# cb_df.to_csv('similarpostsfeatureset.csv')